In [3]:
import json
from pprint import pprint
from typing import Literal, List

import diskcache as dc
import requests
from deepl import Translator
from pydantic import BaseModel
from tqdm import tqdm
from collections import defaultdict

from config import TLCPaths
from data_loading import load_tlc_samples
from models import Match, SampleCollection, Annotation


In [2]:
annotations_file_path = TLCPaths.project_data_path.joinpath("samples_to_annotate.json")
validations_file_path = TLCPaths.project_data_path.joinpath("samples_to_validate.json")
# load the two json files 
with open(annotations_file_path, "r") as f:
    annotation_samples = [Match.parse_raw(x) for x in json.load(f)]
with open(validations_file_path, "r") as f:
    validation_samples = [Match.parse_raw(x) for x in json.load(f)]

print(f"{len(annotation_samples)=}")
print(f"{len(validation_samples)=}")


len(annotation_samples)=2146
len(validation_samples)=5244


In [12]:
print(len([x for x in annotation_samples if x.mention.annotation.type == "LAY"]))
print(len([x for x in annotation_samples if x.mention.annotation.type == "TECH"]))


1460
686


In [8]:
# save json with unique terms for annotation
unique_annotations = defaultdict(list)
for term in annotation_samples:
    if term.mention.annotation.synonyms:
        unique_annotations[term.mention.annotation.get_mention()].append(term.mention.annotation.synonyms[0])


In [9]:
for mention, synonyms in unique_annotations.items():
    synonyms = set(synonyms)
    if len(synonyms) > 1:
        print(mention)
        print(synonyms)
        print()

Gallensteine
{'Cholelith', 'Cholelithiasis'}

Herzklopfen
{'Tachykardie', 'Palpitationen'}

Nierenkolik
{'Wehenartige Nierenschmerzen', 'wehenartige Nierenschmerzen'}

Magen Darm Grippe
{'Gastroenteritis', 'Norovirus-Gastroenteritis'}

niedriger Blutdruck
{'Hypotonie', 'Hypotonus'}

Nierenentzündung
{'Nephritis', 'Pyelonephritis'}

nierenentzündungen
{'Nephritis', 'Nephritiden'}

Blinddarmdurchbruch
{'Perforierte Appendizitis', 'Blinddarmperforation'}

Herzstolpern
{'Extrasystolen', 'Extrasystole', 'Herzrhythmusstörungen'}

Dialyse
{'Blutreinigungsverfahren', 'Nierenersatzverfahren'}

rote Urin
{'Hämaturie', 'sichtbares Blut im Urin', 'Makrohämaturie'}

Mikrohämaturie
{'Laien: ein wenig Blut im Urin', 'Laienbegriff: etwas Blut im Urin', 'etwas Blut im Urin', 'Nicht sichtbares Blut im Urin', 'nicht sichtbares Blut im Urin'}

Nierenkoliken
{'Wehenartige Nierenschmerzen', 'wehenartige Nierenschmerzen'}

Schwitzen
{'Perspiratio sensibilis', 'Transpiration', 'Diaphorese', 'Hydrosis', 'trans